In [8]:
import csv, sqlite3
import pandas as pd
import sqlalchemy as db
import numpy as np
from app.core.http_engine import Request
from app.core.geodata import GeoLocData
import requests
import json

In [ ]:
db_url = "mysql+pymysql://root:secret@db:3306/my_db"
def db_connect(db_url:str):
  try:
    engine = db.create_engine(db_url)    
    metadata = db.MetaData(bind=engine)
    metadata.reflect(only=['test_table'])
    test_table=metadata.tables['test_table']    
    print(f"Connected:\n {test_table}")
    return engine

  except Exception as ex:
          print('Connection could not be made due to the following error: \n', ex)

engine = db_connect(db_url=db_url)

In [2]:
location_name = 'São José dos Campos'
type = "bar"
radius = "50000"
lat = -23.2198396
lon = -45.8915658
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-23.2198396%2C-45.8915658&radius={radius}&type={type}&keyword=cruise&key={google_api_key}"
url2 = f"https://maps.googleapis.com/maps/api/geocode/json?address={location_name}&key={google_api_key}"
url3 = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={location_name}&inputtype=textquery&fields=geometry&key={google_api_key}"
url4 = f"https://maps.googleapis.com/maps/api/place/textsearch/json?&location={lat}%2C{lon}&radius={radius}&type={type}&key={google_api_key}"


In [ ]:
x,y =my_location.build_search_grid(upper_right=[b_box['northeast']['lat'], b_box['northeast']['lng']], lower_left=[b_box['southwest']['lat'], b_box['southwest']['lng']], resolution=100)
print(x,y)


In [ ]:
pd.DataFrame.from_records()(data_dict['SJC'])